In [1]:
%load_ext rustdef

load rustdef


In [2]:
%rustdef depends ndarray

In [3]:
%%rustdef --force-rebuild
use rustdef::numpy::Array;

// immutable example
#[pyfunction]
fn axpy<'a>(a: f64, x: Array<'a, f64>, y: Array<'a, f64>) -> PyResult<Array<'a, f64>> {
    let x = x.to_array();
    let y = y.to_array();
    let z = a * x + y;
    println!("{}", a);
    Ok(z.into())
}

Building..
🔗 Found pyo3 bindings
🐍 Found CPython 3.7m at python3.7
   Compiling rustdef v0.1.0 (/Users/ryosuke.kamesawa/Develop/rustdef/examples/.rustdef/rustdef)
   Compiling rustdef_cell_90ca76e0b1b4a717af6b5fc54e3b7d961fa54e7e v0.1.0 (/Users/ryosuke.kamesawa/Develop/rustdef/examples/.rustdef/rustdef_cell_90ca76e0b1b4a717af6b5fc54e3b7d961fa54e7e)
    Finished release [optimized] target(s) in 6.08s==================> ] 79/80
📦 Built wheel for CPython 3.7m to /Users/ryosuke.kamesawa/Develop/rustdef/examples/.rustdef/target/wheels/rustdef_cell_90ca76e0b1b4a717af6b5fc54e3b7d961fa54e7e-0.1.0-cp37-cp37m-macosx_10_7_x86_64.whl


In [4]:
import numpy as np

In [5]:
axpy(2.4, np.array([1.0, 3.2, 2.9]), np.array([2.3, 10.9, -2]))

2.4


array([ 4.7 , 18.58,  4.96])

In [6]:
%%rustdef
use rustdef::numpy::Array;

// immutable example
#[pyfunction]
fn axpy<'a>(a: f64, x: Array<'a, f64>, y: Array<'a, f64>) -> PyResult<Array<'a, f64>> {
    let x = x.to_array();
    let y = y.to_array();
    let z = a * x + y;
    Ok(z.into())
}

// mutable example
#[pyfunction]
fn mult_inline<'a>(a: f64, x: Array<'a, f64>) -> PyResult<()> {
    let mut x = x.as_mut_array()?;
    x *= a;
    Ok(())
}

Use previous build


In [7]:
x = np.random.rand(10)
y = np.random.rand(10)
x, y

(array([0.57309634, 0.41947385, 0.03517951, 0.48253452, 0.69774727,
        0.26808764, 0.81478835, 0.67665616, 0.03087333, 0.58134473]),
 array([0.66893296, 0.46087589, 0.82905176, 0.89316878, 0.27341331,
        0.32972173, 0.71045832, 0.65304343, 0.74426215, 0.37519448]))

In [8]:
axpy(3.0, x, y)

array([2.38822197, 1.71929743, 0.9345903 , 2.34077232, 2.36665513,
       1.13398464, 3.15482336, 2.68301192, 0.83688214, 2.11922865])

In [9]:
mult_inline(12.0, x)

In [10]:
x

array([6.87715607, 5.0336862 , 0.42215416, 5.79041419, 8.37296727,
       3.21705164, 9.77746017, 8.11987397, 0.37047993, 6.97613671])